Do all of this with Beautiful Soup?

In [2]:
import pandas as pd
import numpy as np

# Fantasy Footballers UDK Rankings

https://www.thefantasyfootballers.com/2023-ultimate-draft-kit/udk-dynasty-startup-rankings/

In [6]:
all = pd.read_csv('Dynasty_Ranking_23_24_All.csv')
qb = pd.read_csv('Dynasty_Ranking_23_24_QB.csv')
rb = pd.read_csv('Dynasty_Ranking_23_24_RB.csv')
wr = pd.read_csv('Dynasty_Ranking_23_24_WR.csv')
te = pd.read_csv('Dynasty_Ranking_23_24_TE.csv') 
rook = pd.read_csv('Dynasty_Ranking_23_24_Rookie.csv') 

In [7]:
# DATA PREP

# select columns wanted
all = all[["Rank", "Name", "Team", "Age"]]

# QB
qb = qb[["Rank", "Name"]]
qb = qb.assign(Pos = 'QB')
qb['Rank'] = qb['Rank'].astype(str)
qb['Pos_Rnk'] = qb['Pos'] + "-" + qb['Rank']

# RB
rb = rb[["Rank", "Name"]]
rb = rb.assign(Pos = 'RB')
rb['Rank'] = rb['Rank'].astype(str)
rb['Pos_Rnk'] = rb['Pos'] + "-" + rb['Rank']

# WR
wr = wr[["Rank", "Name"]]
wr = wr.assign(Pos = 'WR')
wr['Rank'] = wr['Rank'].astype(str)
wr['Pos_Rnk'] = wr['Pos'] + "-" + wr['Rank']

# TE
te = te[["Rank", "Name"]]
te = te.assign(Pos = 'TE')
te['Rank'] = te['Rank'].astype(str)
te['Pos_Rnk'] = te['Pos'] + "-" + te['Rank']

# Rookie
rook = rook[["Rank", "Name"]]
rook = rook.rename(columns = {'Rank': 'Rook_Rnk'})
rook['Rook_Rnk'] = rook['Rook_Rnk'].astype(str)

# Join data
comb = pd.concat([qb, rb, wr, te])
comb2 = pd.merge(comb, rook, on = 'Name', how = 'left')
all_fnl = pd.merge(all, comb2, on = 'Name', how = 'left')

# Select cols and final clean
all_fnl = all_fnl[["Rank_x", "Name", "Team", "Age", "Pos", "Pos_Rnk", "Rook_Rnk"]]
all_fnl = all_fnl.rename(columns = {'Rank_x': 'Rank'})
all_fnl = all_fnl.replace(np.nan, '', regex=True)

In [135]:
# Write csv
all_fnl.to_csv('Dynasty_FF_Rankings_23_24_Clean.csv', index = False)

# Sleeper ADP

https://www.ftnfantasy.com/nfl/tools/sleeper-fantasy-football-adp

In [4]:
# Read in adp data
adp = pd.read_csv('sleeper_adp_23_24 8.15.csv') 

In [8]:
# remove cols
adp = adp.drop(columns = ['Redraft SF ADP', 'Dynasty SF ADP', 'IDP ADP'])

# Dynasty PPR
adp_dyn_ppr = adp[["Player", "Team", "Position", "Dynasty PPR ADP"]].sort_values("Dynasty PPR ADP")

# remove nan
adp_dyn_ppr = adp_dyn_ppr[adp_dyn_ppr['Dynasty PPR ADP'].notna()]

# create dynasty ppr rank
adp_dyn_ppr['ADP_Rnk'] = adp_dyn_ppr['Dynasty PPR ADP'].rank(ascending=True).astype(np.int64)

# create position rankings
adp_dyn_ppr['Pos_Rnk'] = adp_dyn_ppr.groupby('Position')['Dynasty PPR ADP'].rank(ascending=True).astype(np.int64)

adp_dyn_ppr['ADP_Pos_Rnk']  = adp_dyn_ppr['Position'] + "-" + adp_dyn_ppr['Pos_Rnk'].astype(str)

# select columns and final cleaning
adp_dyn_ppr = adp_dyn_ppr.rename(columns = {'ADP_Rnk': 'ADP_Rank', 'Player': 'Name', 'Position': 'Pos'})

adp_dyn_ppr = adp_dyn_ppr[["ADP_Rank", "Name", "Team", "Pos", "ADP_Pos_Rnk"]]

# get age and rookie info
age_rook = all_fnl[["Name", "Age", "Rook_Rnk"]]

# prep
age_rook.loc[age_rook['Rook_Rnk'] != '', 'Rook'] = 'True' 
age_rook = age_rook.replace(np.nan, '', regex=True)
age_rook = age_rook[["Name", "Age", "Rook"]]

KeyError: "['Redraft SF ADP', 'Dynasty SF ADP', 'IDP ADP'] not found in axis"

In [133]:
# join age and rookie info
adp_comb = pd.merge(adp_dyn_ppr, age_rook, on = 'Name', how = 'left')
adp_comb = adp_comb.replace(np.nan, '', regex=True)
adp_comb['Rook_Rnk'] = adp_comb.groupby('Rook')['ADP_Rank'].rank(ascending=True)

# get rookie rank to be able to join back
just_rook = adp_comb[(adp_comb.Rook == "True")]
just_rook = just_rook[["Name", "Rook_Rnk"]]

# remove a coup[le cols before joining back]
adp_comb = adp_comb.drop(columns = ['Rook_Rnk', 'Rook'])

# join back rookie
adp_rook_comb = pd.merge(adp_comb, just_rook, on = 'Name', how = 'left')
adp_rook_comb = adp_rook_comb.replace(np.nan, '', regex=True)

# move age column
move_age = adp_rook_comb.pop("Age")
# insert column with insert(location, column_name, column_value)
adp_rook_comb.insert(3, "Age", move_age)

In [136]:
# Write csv
adp_rook_comb.to_csv('Sleeper_PPR_Dyn_ADP_23_24_Clean.csv', index = False)

# ESPN API / Scrape

Test to get historical league resultsd and try to determine keepers

leagueId = 1057699
seasonId = 2023 (whatever year you are looking for)



# 2022 example
Rosters = https://fantasy.espn.com/football/league/rosters?leagueId=1057699&seasonId=2022

Draft = https://fantasy.espn.com/football/league/draftrecap?seasonId=2022&leagueId=1057699

# League of Goo

Redraft with Staley

In [9]:
# Dynasty PPR
adp_redraft = adp[["Player", "Team", "Position", "Redraft Half PPR ADP"]].sort_values("Redraft Half PPR ADP")

# remove nan
adp_redraft = adp_redraft[adp_redraft['Redraft Half PPR ADP'].notna()]

# create dynasty ppr rank
adp_redraft['ADP_Rnk'] = adp_redraft['Redraft Half PPR ADP'].rank(ascending=True).astype(np.int64)

# create position rankings
adp_redraft['Pos_Rnk'] = adp_redraft.groupby('Position')['Redraft Half PPR ADP'].rank(ascending=True).astype(np.int64)

adp_redraft['ADP_Pos_Rnk']  = adp_redraft['Position'] + "-" + adp_redraft['Pos_Rnk'].astype(str)

# select columns and final cleaning
adp_redraft = adp_redraft.rename(columns = {'ADP_Rnk': 'ADP_Rank', 'Player': 'Name', 'Position': 'Pos'})

adp_redraft = adp_redraft[["ADP_Rank", "Name", "Team", "Pos", "ADP_Pos_Rnk"]]

In [10]:
# Write csv
adp_redraft.to_csv('Sleeper_Redraft_ADP_23_24_Clean.csv', index = False)

In [15]:






# get age and rookie info
age_rook = all_fnl[["Name", "Age", "Rook_Rnk"]]

# prep
age_rook.loc[age_rook['Rook_Rnk'] != '', 'Rook'] = 'True' 
age_rook = age_rook.replace(np.nan, '', regex=True)
age_rook = age_rook[["Name", "Age", "Rook"]]